## 1. Loading the data
Take reference seq and use translate tool to get sequence.

https://www.mavedb.org/#/score-sets/urn:mavedb:00000086-a-1 

In [20]:
wild_type_seq = "MSIQHFRVAL IPFFAAFCLP VFAHPETLVK VKDAEDQLGA RVGYIELDLN SGKILESFRP EERFPMMSTF KVLLCGAVLS RVDAGQEQLG RRIHYSQNDL VEYSPVTEKH LTDGMTVREL CSAAITMSDN TAANLLLTTI GGPKELTAFL HNMGDHVTRL DRWEPELNEA IPNDERDTTM PAAMATTLRK LLTGELLTLA SRQQLIDWME ADKVAGPLLR SALPAGWFIA DKSGAGERGS RGIIAALGPD GKPSRIVVIY TTGSQATMDE RNRQIAEIGA SLIKHW".replace(" ", "")

In [21]:
import Bio.SeqUtils as bio
import pandas as pd
import re

csv = pd.read_csv("E6.csv", usecols=["hgvs_pro","score"])
X_seqs = []
y = []
for index, row in csv.iterrows():
    end = row["hgvs_pro"][-3:]
    position = int(re.findall(r'\d+', row["hgvs_pro"])[0])-1
    mutant_seq = wild_type_seq
    mutant_seq = list(mutant_seq)
    mutant_seq[position] = bio.IUPACData.protein_letters_3to1[end]
    if not pd.isna(row["score"]):
        X_seqs.append(mutant_seq)
        y.append(float(row["score"]))

In [22]:
from sklearn.preprocessing import OneHotEncoder
import torch
enc = OneHotEncoder()
X_onehot = enc.fit_transform(X_seqs).toarray()
X_tensor = torch.tensor(X_onehot, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).long()

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2)

In [24]:
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import pytorch_lightning as pl

class MLP(pl.LightningModule):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.loss_fn(y_pred, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

# Assuming you have X_train, y_train, X_test, and y_test tensors
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

input_size = X_train.size(1)
output_size = torch.unique(y_train).size(0)
hidden_size = 64  # Adjust as needed

model = MLP(input_size, hidden_size, output_size)

trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, train_loader, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/anthosbz/Documents/Learning_ML/First_ML/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.

  | Name    | Type             | Params
---------------------------------------------
0 | fc1     | Linear           | 329 K 
1 | fc2     | Linear           | 4.2 K 
2 | fc3     | Linear           | 65    
3 | relu    | ReLU             | 0     
4 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
333 K     Trainable params
0         Non-trainable params
333 K     Total params
1.336     Total estimated model params size (MB)
/home/anthosbz/Documents/Learning_ML/First_ML/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does 

Epoch 0:   0%|          | 0/61 [00:00<?, ?it/s] 

Epoch 9: 100%|██████████| 61/61 [00:00<00:00, 170.10it/s, v_num=2]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 61/61 [00:00<00:00, 166.65it/s, v_num=2]
